In [1]:
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [7]:
mic_coords = pd.read_excel("data/mic_coords_final.xlsx", index_col=0)
distances = pd.read_excel("data/miscellaneous_positions.xlsx", index_col=0)

# remove NaNs
distances = distances.drop(index=[12, 13])
print(distances)

def get_mic_coords(name, df, x, y, z):
    filtered_df = df[df['Mic_Index'] == name]
    coordinates = filtered_df[[x, y, z]].to_numpy()
    return coordinates[0]



position_coords_default = []
for name in distances["point 2"]:
    coord = get_mic_coords(name, mic_coords, "X", "Y", "Z")
    position_coords_default.append(coord)

position_coords_optimized = []
for name in distances["point 2"]:
    coord = get_mic_coords(name, mic_coords, "X_optimized", "Y_optimized", "Z_optimized")
    position_coords_optimized.append(coord)



                        point 1 point 2  distance
0   basson lower mounting point      B3     0.505
1   basson lower mounting point     B10     0.740
2   basson lower mounting point      B6     0.684
3        basson bell top middle     C12     0.484
4        basson bell top middle     C16     0.560
5        basson bell top middle     C17     0.493
6   camera pos 1 (lense center)      B5     0.826
7   camera pos 1 (lense center)      C6     0.837
8   camera pos 1 (lense center)     A10     1.490
9                  camera pos 2      A7     0.470
10                 camera pos 2     B13     0.835
11                 camera pos 2     C14     0.926


In [10]:
def triangulate_point(list_coords, df_distances, start):
    P1 = list_coords[start]
    P2 = list_coords[start+1]
    P3 = list_coords[start+2]

    d1 = df_distances["distance"][start]
    d2 = df_distances["distance"][start+1]
    d3 = df_distances["distance"][start+2]

    # Build the system of linear equations
    A = 2 * np.array([
        [P2[0] - P1[0], P2[1] - P1[1], P2[2] - P1[2]],
        [P3[0] - P1[0], P3[1] - P1[1], P3[2] - P1[2]]
        ])

    b = np.array([
        d1**2 - d2**2 + np.dot(P2, P2) - np.dot(P1, P1),
        d1**2 - d3**2 + np.dot(P3, P3) - np.dot(P1, P1)
        ])

    # Solve for x, y, z using least squares
    xyz = np.linalg.lstsq(A, b, rcond=-1)[0]

    # Compute z-coordinate explicitly (if needed)
    z_squared = d1**2 - np.dot(xyz - P1, xyz - P1)
    print("z_squared = ", z_squared) # complex means bad :(
    z = np.sqrt(z_squared) if z_squared >= 0 else None

    print("Coordinates of the unknown point:")
    print(f"x: {xyz[0]}, y: {xyz[1]}, z: ±{z}")
    
    try:
        coord = np.array([[xyz[0], xyz[1], z], [xyz[0], xyz[1], -z]])
        return coord
    
    except:
        print("No return coordinate")


items = ["basson_low", "basson_high", "camera_1", "camera_2"]
start_points = [0, 3, 6, 9]


print("OPTIMIZED VALUES")
print("")
for start, item in zip(start_points, items):
    print(item)
    _ = triangulate_point(position_coords_optimized, distances, start)
    print("")

print("")
print("DEFAULT CALCULATED VALUES")
print("")
for start, item in zip(start_points, items):
    print(item)
    _ = triangulate_point(position_coords_default, distances, start)
    print("")


OPTIMIZED VALUES

basson_low
z_squared =  -1.819024328491085
Coordinates of the unknown point:
x: 0.00223031678506469, y: 1.0053779533482348, z: ±None
No return coordinate

basson_high
z_squared =  0.12382873765795283
Coordinates of the unknown point:
x: 0.5257270504102248, y: -0.05682104576017928, z: ±0.3518930770247588

camera_1
z_squared =  0.132899785070402
Coordinates of the unknown point:
x: 0.6986960438917849, y: 0.7016823714541304, z: ±0.36455422788715813

camera_2
z_squared =  -2.5361377215643572
Coordinates of the unknown point:
x: -0.20667685426861693, y: 1.107267247639946, z: ±None
No return coordinate


DEFAULT CALCULATED VALUES

basson_low
z_squared =  -1.6732682979236642
Coordinates of the unknown point:
x: 5.551115123125783e-17, y: 1.028945827439887, z: ±None
No return coordinate

basson_high
z_squared =  0.12382873765795283
Coordinates of the unknown point:
x: 0.519369940308797, y: 0.0, z: ±0.3518930770247588

camera_1
z_squared =  0.1774531162692473
Coordinates of the

In [4]:
from _mic_position_functions import calculate_distance

_ = calculate_distance(mic_coords, "B10", "B3", x_col="X_optimized", y_col="Y_optimized", z_col="Z_optimized")
_ = calculate_distance(mic_coords, "B6", "B3", x_col="X_optimized", y_col="Y_optimized", z_col="Z_optimized")
_ = calculate_distance(mic_coords, "B10", "B6", x_col="X_optimized", y_col="Y_optimized", z_col="Z_optimized")

0.703 = distance B10 to B3
0.737 = distance B6 to B3
0.713 = distance B10 to B6


In [5]:
bassoon_high = triangulate_point(position_coords_optimized, distances, 3)[0]
camera_1 = triangulate_point(position_coords_optimized, distances, 6)[0]


fig = px.scatter_3d(mic_coords, x='X_optimized', y='Y_optimized', z='Z_optimized', text='Mic_Index', title='3D Scatter Plot', color='Plane', width=1000, height=1000)


# Add the single point to the existing figure
fig.add_trace(go.Scatter3d(
    x=[bassoon_high[0]],  # x-coordinate of the single point
    y=[bassoon_high[1]],  # y-coordinate of the single point
    z=[bassoon_high[2]],  # z-coordinate of the single point
    mode='markers+text',  # Include markers and optional text
    marker=dict(size=10, color='red'),  # Customize the marker style
    text=['bassoon_high'],  # Optional text label for the point
    name='bassoon_high'  # Legend entry
))

# Add the single point to the existing figure
fig.add_trace(go.Scatter3d(
    x=[camera_1[0]],  # x-coordinate of the single point
    y=[camera_1[1]],  # y-coordinate of the single point
    z=[camera_1[2]],  # z-coordinate of the single point
    mode='markers+text',  # Include markers and optional text
    marker=dict(size=10, color='red'),  # Customize the marker style
    text=['camera_1'],  # Optional text label for the point
    name='camera_1'  # Legend entry
))


fig.update_layout(scene=dict(
    xaxis=dict(range=[1.5, 0]), # x-Achse umdrehen - "linke hand regel"
))

# Plot anzeigen
fig.show()

z_squared =  0.12382873765795283
Coordinates of the unknown point:
x: 0.5257270504102248, y: -0.05682104576017928, z: ±0.3518930770247588
z_squared =  0.132899785070402
Coordinates of the unknown point:
x: 0.6986960438917849, y: 0.7016823714541304, z: ±0.36455422788715813
